# Exploring Whitehouse Visitors

> Burcin Sarac<br/>
> MSc. Business Analytics FT18<br/>
> Department of Management Science and Technology<br/>
> Athens University of Economics and Business

In this project I used PySpark to answer following questions by using the White House visitors records data released on Obama administration period, gotten from <https://obamawhitehouse.archives.gov/goodgovernment/tools/visitor-records>.

1. Who are the top 20 visitors?

2. Who are the top 20 visitees?

3. Who are the top 20 visitor-visitee pairs?

4. What were the top 20 most busy days?

5. What where the top 20 most busy months-years?

6. What was the order of popularity of days of week for visits? 

7. What was the order of popularity of months for visits?

In [1]:
import os
os.chdir("E:/dersler/big data systems/Assignment_2")

Before running Spark via Python on $Windows$ $10$, there are several steps that needs to be taken care of which are mentioned below; 

* Downloaded Spark from the [Spark web site](https://spark.apache.org/).

* Uncompressed it and placed it into a directory (mine is, `E:/Python/spark/spark2.4.2`).

* Added the directory to the path with naming it as `SPARK_HOME`.

* Downloaded winutils.exe binary from https://github.com/steveloughran/winutils repository.

* Saved winutils.exe binary to a directory of my choice, mine is `c:\hadoop\bin`

* Again added the directory with setting name as`HADOOP_HOME` (without bin).

* Created `C:\tmp\hive` directory.

* Executed command below from cmd as administrator to give access permission to the spark;
`winutils.exe chmod -R 777 C:\tmp\hive`

* Also installed pyspark and findspark packages via pip from terminal.

* And for setting the scene; 
    * Typed `pyspark` from terminal. 
    * And created connection and a new SparkContext with codes below to interact with Spark.

In [2]:
import findspark
findspark.init("E:/Python/spark/spark2.4.2")
from pyspark.sql import SparkSession

spark =  SparkSession.builder.appName("Whitehouse_visitors").getOrCreate()

sc = spark.sparkContext

The csv files had downloaded as separate files for every year. After uncompressed them, created a new folder called "visitors" and added all csv files into it. Then, read the files by executing the command below, which also force the program to read first row as header. With the following commands I tried to view the schema of the DataFrame.

In [3]:
parsed = spark.read.\
    option("header", "true").\
    option("inferSchema", "true").\
    csv("visitors")

In [4]:
parsed.count()

3727273

In [7]:
parsed.show()

+--------+---------+-------+------+------+-----------+-------------+-----+-------------+----+--------------+---------------+--------------+----------------+------------+--------------+----+-------------+---------------+----------------+-----------------+-----------+------------+----------------+-----------------+-----------+--------------------+------------+
|NAMELAST|NAMEFIRST|NAMEMID|   UIN|BDGNBR|ACCESS_TYPE|          TOA|  POA|          TOD| POD|APPT_MADE_DATE|APPT_START_DATE| APPT_END_DATE|APPT_CANCEL_DATE|Total_People|LAST_UPDATEDBY|POST|LastEntryDate|TERMINAL_SUFFIX|visitee_namelast|visitee_namefirst|MEETING_LOC|MEETING_ROOM|CALLER_NAME_LAST|CALLER_NAME_FIRST|CALLER_ROOM|         Description|Release Date|
+--------+---------+-------+------+------+-----------+-------------+-----+-------------+----+--------------+---------------+--------------+----------------+------------+--------------+----+-------------+---------------+----------------+-----------------+-----------+------------

In [8]:
parsed.printSchema()

root
 |-- NAMELAST: string (nullable = true)
 |-- NAMEFIRST: string (nullable = true)
 |-- NAMEMID: string (nullable = true)
 |-- UIN: string (nullable = true)
 |-- BDGNBR: string (nullable = true)
 |-- ACCESS_TYPE: string (nullable = true)
 |-- TOA: string (nullable = true)
 |-- POA: string (nullable = true)
 |-- TOD: string (nullable = true)
 |-- POD: string (nullable = true)
 |-- APPT_MADE_DATE: string (nullable = true)
 |-- APPT_START_DATE: string (nullable = true)
 |-- APPT_END_DATE: string (nullable = true)
 |-- APPT_CANCEL_DATE: string (nullable = true)
 |-- Total_People: string (nullable = true)
 |-- LAST_UPDATEDBY: string (nullable = true)
 |-- POST: string (nullable = true)
 |-- LastEntryDate: string (nullable = true)
 |-- TERMINAL_SUFFIX: string (nullable = true)
 |-- visitee_namelast: string (nullable = true)
 |-- visitee_namefirst: string (nullable = true)
 |-- MEETING_LOC: string (nullable = true)
 |-- MEETING_ROOM: string (nullable = true)
 |-- CALLER_NAME_LAST: string (

Since dataframe includes different data types, I created a schema first according to provided data explanation and read the dataset again this time by using schema command, however it did convert all observations into null data, so I decided not to follow that path.

Spark needs to re-read and parse again the data everytime we asked for an action.

Since I will be using the data again and again, it makes sense to cache the parse results, so that they are already available and I do not need to re-read and re-parse them.

By using `cache()`, Spark will try to persist the data in memory in each partition node.

In [10]:
parsed.cache()

DataFrame[NAMELAST: string, NAMEFIRST: string, NAMEMID: string, UIN: string, BDGNBR: string, ACCESS_TYPE: string, TOA: string, POA: string, TOD: string, POD: string, APPT_MADE_DATE: string, APPT_START_DATE: string, APPT_END_DATE: string, APPT_CANCEL_DATE: string, Total_People: string, LAST_UPDATEDBY: string, POST: string, LastEntryDate: string, TERMINAL_SUFFIX: string, visitee_namelast: string, visitee_namefirst: string, MEETING_LOC: string, MEETING_ROOM: string, CALLER_NAME_LAST: string, CALLER_NAME_FIRST: string, CALLER_ROOM: string, Description: string, Release Date: string]

In [13]:
parsed.filter(parsed.APPT_CANCEL_DATE.isNotNull()).groupBy("APPT_CANCEL_DATE").count().show()

+----------------+-----+
|APPT_CANCEL_DATE|count|
+----------------+-----+
|   4/23/12 15:41|   40|
|   9/26/12 15:10|    1|
|    1/30/12 9:45|    1|
|10/26/2010 10:41|    2|
|11/15/2010 18:11|    1|
|   2/1/2011 9:05|    5|
|  2/3/2011 17:35|    1|
| 4/14/2010 10:07|    1|
|        3/7/2010|    3|
|  6/8/2011 10:54|    1|
|  10/11/11 11:31|    1|
|11/21/2011 10:11|    1|
|10/15/2013 16:04|    1|
| 3/27/2013 13:03|    1|
| 1/15/2014 10:47|    1|
|     41675.74465|    1|
|   3/19/12 17:28|    1|
| 10/8/2010 17:11|    1|
| 1/18/2011 14:13|    3|
| 7/28/2010 12:30|   45|
+----------------+-----+
only showing top 20 rows



With the command above I checked if were there any cancelled appointments and it seems there were. I used this information at the following questions. 

# Q-1

1. Who are the top 20 visitors?

For solving this question, first I filtered observations who cancelled their appointment by using `APPT_CANCEL_DATE` column data. Then I groupped by data into `NAMELAST` and `NAMEFIRST` columns and count. With `orderBy` command, I ordered data and it shows first 20 highest counted rows. 

In [14]:
parsed.\
    filter(parsed.APPT_CANCEL_DATE.isNull()).\
    groupBy("NAMELAST", "NAMEFIRST").\
    count().\
    orderBy("count", ascending=False).\
    show()

+------------+---------+-----+
|    NAMELAST|NAMEFIRST|count|
+------------+---------+-----+
|        Hash|  Michael|  726|
|    Tavenner|  Marilyn|  519|
|        Hoff|    James|  482|
|BrooksLaSure| Chiquita|  410|
|     Levitis|    Jason|  384|
|       Borzi|  Phyllis|  368|
|    Fontenot|   Yvette|  350|
|      Khalid|   Aryana|  341|
|        Mann|  Cynthia|  323|
|      Werner|   Sharon|  310|
|        Choe|  Kenneth|  305|
|       Smith|  Michael|  304|
|       Jones|   Daniel|  303|
|      Turner|      Amy|  302|
|    Morrison|    Helen|  295|
|  Livingston|Catherine|  295|
|       Lewis|     Caya|  292|
|     Kronick|  Richard|  292|
|     Maguire|   Daniel|  290|
|     Schultz|  William|  283|
+------------+---------+-----+
only showing top 20 rows



# Q-2

2. Who are the top 20 visitees?

For this question, again I began with filtered observations who cancelled their appointment by using `APPT_CANCEL_DATE` column data, additionally I also filter observations under `visitee_namelast` and `visitee_namefirst` data, which did not refer to a visitee name. I also dropped null data by using `na.drop` command. Then I groupped by data into  `visitee_namelast` and `visitee_namefirst` columns and count. With `orderBy` command, I ordered data and it shows first 20 highest counted visitee name and surname combinations.  

In [15]:
parsed.\
    filter((parsed.visitee_namelast != "OFFICE")&(parsed.visitee_namelast != "/")&(parsed.visitee_namefirst != "OFFICE")&
          (parsed.visitee_namelast != "and")).\
    filter(parsed.APPT_CANCEL_DATE.isNull()).\
    groupBy("visitee_namelast", "visitee_namefirst").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+----------------+-----------------+-----+
|visitee_namelast|visitee_namefirst|count|
+----------------+-----------------+-----+
|         Lierman|             Kyle|14453|
|         Lambrew|           Jeanne|14127|
|         DOEBLER|              MAX| 5962|
|      MCCULLOUGH|         VICTORIA| 5720|
|        Sperling|             Gene| 4327|
|          NELSON|             GREG| 4278|
|           Hogan|            Nancy| 4226|
|          DuBois|           Joshua| 4198|
|        Raghavan|           Gautam| 3985|
|          Foster|          Heather| 3900|
|        Monteiro|             Paul| 3885|
|            Park|             Todd| 3657|
|      Richardson|            Karen| 3546|
|         McKalip|             Doug| 3338|
|         doebler|              max| 3259|
|           Kelly|           Lauren| 3216|
|             Cho|           Ronnie| 3189|
|            FENN|            SARAH| 3149|
|           kelly|           lauren| 3120|
|        monteiro|             paul| 3092|
+----------

# Q-3

3. Who are the top 20 visitor-visitee pairs?

For this question, again I began with filtered observations who cancelled their appointment by using APPT_CANCEL_DATE column data, additionally I also filter observations under visitee_namelast and visitee_namefirst data, which did not refer to a visitee name. I also dropped null data by using na.drop command. But this time, I groupped by data into visitee_namelast and visitee_namefirst columns together with `NAMELAST` and `NAMEFIRST` columns and count. With orderBy command, I ordered data and it shows first 20 highest counted visitee - visitor pairs with name and surname combinations.

In [16]:
parsed.\
    filter((parsed.visitee_namelast != "OFFICE")&(parsed.visitee_namelast != "/")&(parsed.visitee_namefirst != "OFFICE")).\
    filter(parsed.APPT_CANCEL_DATE.isNull()).\
    groupBy("visitee_namelast","visitee_namefirst", "NAMELAST", "NAMEFIRST").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+----------------+-----------------+------------+---------+-----+
|visitee_namelast|visitee_namefirst|    NAMELAST|NAMEFIRST|count|
+----------------+-----------------+------------+---------+-----+
|         Lambrew|           Jeanne|        Hash|  Michael|  515|
|            Hoff|           Joanne|        Hoff|    James|  387|
|         Lambrew|           Jeanne|    Tavenner|  Marilyn|  379|
|         Lambrew|           Jeanne|BrooksLaSure| Chiquita|  324|
|         Lambrew|           Jeanne|    Fontenot|   Yvette|  292|
|         Lambrew|           Jeanne|      Khalid|   Aryana|  283|
|         Lambrew|           Jeanne|     Levitis|    Jason|  280|
|         Lambrew|           Jeanne|        Mann|  Cynthia|  269|
|         Lambrew|           Jeanne|     Kronick|  Richard|  252|
|         Lambrew|           Jeanne|        Choe|  Kenneth|  242|
|         Lambrew|           Jeanne|  Livingston|Catherine|  240|
|         Lambrew|           Jeanne|    Morrison|    Helen|  235|
|         

# Q-4

4. What were the top 20 most busy days?

This time, for the following questions, I was needed to convert `APPT_MADE_DATE` column into timestamp datatype. Since the dataset was not clean and one typed organized, fitting a previously created Schema when reading csv files did not performed as expected and produced null values for all columns. So I decided to convert only the required date column and separate them from the whole dataframe for answering following questions. 

So I first imported `unix_timestamp`and `from_unixtime`functions from `pyspark.sql.functions` library. With the helps of these funtions, I created a new dataframe only includes `APPT_MADE_DATE` column and its converted version named `appt_m_date` by using following commands. 

After converting related column, I first checked if it can read the timestamp data as expected by checking days of week. And then for answering the question, I let the command count the days by appearance from converted `appt_m_date` column and order them from highest to lowest. And I also dropped null data. 

In [17]:
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import from_unixtime

parsed2 = parsed.select('APPT_MADE_DATE','Total_People', 
                        from_unixtime(unix_timestamp('APPT_MADE_DATE', 'MM/dd/yy')).alias('appt_m_date'))

parsed2.show()

+--------------+------------+-------------------+
|APPT_MADE_DATE|Total_People|        appt_m_date|
+--------------+------------+-------------------+
|  2/23/12 0:00|           8|2012-02-23 00:00:00|
|  2/23/12 0:00|           8|2012-02-23 00:00:00|
|   5/7/12 0:00|           1|2012-05-07 00:00:00|
|  5/21/12 0:00|           1|2012-05-21 00:00:00|
|  9/14/12 0:00|          43|2012-09-14 00:00:00|
|   6/7/12 0:00|        null|2012-06-07 00:00:00|
|   6/7/12 0:00|        null|2012-06-07 00:00:00|
|   6/7/12 0:00|        null|2012-06-07 00:00:00|
|  9/25/12 0:00|         268|2012-09-25 00:00:00|
|  9/25/12 0:00|         268|2012-09-25 00:00:00|
|  9/25/12 0:00|         268|2012-09-25 00:00:00|
|  12/6/12 0:00|         254|2012-12-06 00:00:00|
|          null|        null|               null|
|  9/17/12 0:00|         291|2012-09-17 00:00:00|
|  9/18/12 0:00|         158|2012-09-18 00:00:00|
|  3/22/12 0:00|         273|2012-03-22 00:00:00|
|  3/23/12 0:00|         273|2012-03-23 00:00:00|


In [18]:
from pyspark.sql.functions import date_format, col

prepped_dataframe = parsed2\
  .na.fill(0)\
  .withColumn("day_of_week", date_format(col("appt_m_date"), "EEEE"))

In [19]:
prepped_dataframe.select("day_of_week").distinct().orderBy("day_of_week").show()

+-----------+
|day_of_week|
+-----------+
|       null|
|     Friday|
|     Monday|
|   Saturday|
|     Sunday|
|   Thursday|
|    Tuesday|
|  Wednesday|
+-----------+



In [21]:
parsed2.\
    groupBy("appt_m_date").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+-------------------+-----+
|        appt_m_date|count|
+-------------------+-----+
|2012-05-08 00:00:00|12301|
|2011-12-06 00:00:00|11572|
|2011-12-07 00:00:00|11335|
|2010-12-15 00:00:00|10556|
|2012-03-19 00:00:00|10312|
|2012-12-05 00:00:00|10059|
|2009-12-08 00:00:00| 9448|
|2009-12-09 00:00:00| 9389|
|2011-10-27 00:00:00| 9379|
|2012-06-21 00:00:00| 9341|
|2013-11-22 00:00:00| 9271|
|2012-12-03 00:00:00| 9252|
|2012-03-20 00:00:00| 9143|
|2013-02-20 00:00:00| 9127|
|2011-10-12 00:00:00| 8990|
|2012-12-06 00:00:00| 8979|
|2012-04-11 00:00:00| 8920|
|2012-03-13 00:00:00| 8837|
|2013-03-07 00:00:00| 8730|
|2012-07-25 00:00:00| 8555|
+-------------------+-----+
only showing top 20 rows



# Q-5

5. What where the top 20 most busy months-years?

This time,I again used same converted data frame but this time I groupped by data by Month and Year pairs, again order them by appearance and dropped the null data. 


In [22]:
prepped_dataframe2 = parsed2\
  .na.fill(0)\
  .withColumn("by_month_year", date_format(col("appt_m_date"), "MMM-YYY"))

In [23]:
prepped_dataframe2.select("by_month_year").distinct().orderBy("by_month_year").show()

+-------------+
|by_month_year|
+-------------+
|         null|
|     Apr-2009|
|     Apr-2010|
|     Apr-2011|
|     Apr-2012|
|     Apr-2013|
|     Aug-2009|
|     Aug-2010|
|     Aug-2011|
|     Aug-2012|
|     Aug-2013|
|     Dec-2008|
|     Dec-2009|
|     Dec-2010|
|     Dec-2011|
|     Dec-2012|
|     Dec-2013|
|     Dec-2014|
|     Feb-2009|
|     Feb-2010|
+-------------+
only showing top 20 rows



In [24]:
prepped_dataframe2.\
    groupBy("by_month_year").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+-------------+------+
|by_month_year| count|
+-------------+------+
|     Dec-2009|112023|
|     Jun-2010|109463|
|     Jul-2012|104254|
|     Mar-2010|103741|
|     Jun-2012| 98570|
|     Oct-2011| 98237|
|     Jun-2011| 97600|
|     Dec-2010| 96908|
|     May-2011| 95355|
|     Mar-2012| 91679|
|     Mar-2011| 91468|
|     Jul-2011| 89092|
|     Dec-2011| 88418|
|     May-2010| 88302|
|     May-2012| 87271|
|     Aug-2011| 83919|
|     Dec-2012| 83144|
|     Apr-2012| 82220|
|     Apr-2011| 81539|
|     Aug-2010| 79755|
+-------------+------+
only showing top 20 rows



# Q-6

6. What was the order of popularity of days of week for visits? 

This time,I again used same converted data frame and same dataframe which I created for the Q4 with the commands above. Then I groupped by data by Days, again order them by appearance and dropped the null data. 


In [25]:
prepped_dataframe.\
    groupBy("day_of_week").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+-----------+------+
|day_of_week| count|
+-----------+------+
|    Tuesday|776124|
|  Wednesday|730959|
|   Thursday|701142|
|     Monday|647961|
|     Friday|647596|
|   Saturday| 99687|
|     Sunday| 40514|
+-----------+------+



# Q-7

7. What was the order of popularity of months for visits?

This time,I again used same converted data frame and follow same path as I used at the previous question, only difference was pointing out months instead of days this time. Then I groupped by data by Months, again order them by appearance and dropped the null data.

In [26]:
prepped_dataframe3 = parsed2\
  .na.fill(0)\
  .withColumn("by_month", date_format(col("appt_m_date"), "MMM"))

In [27]:
prepped_dataframe3.\
    groupBy("by_month").\
    count().\
    orderBy("count", ascending=False).\
    na.drop().show()

+--------+------+
|by_month| count|
+--------+------+
|     Dec|457258|
|     Mar|385443|
|     Nov|340281|
|     Oct|339934|
|     Jun|325137|
|     May|292951|
|     Jul|270481|
|     Apr|266545|
|     Feb|254596|
|     Sep|251530|
|     Aug|243136|
|     Jan|216691|
+--------+------+

